In [1]:
# Import necessary libraries
import torch
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

# Import custom modules
from common.chroma_db import ChromaDBSearcher
from common.chat_model import ModelQA
from common.cache import ModelCache

In [2]:
# 2. Initialize model cache and load model
model_cache = ModelCache()

# Model selection
model_id = 'meta-llama/Llama-2-7b-chat-hf'  # Example model ID for the chatbot

# Check if the model is already cached to avoid reloading it
model_qa = model_cache.get(model_id)
if model_qa is None:
    # If the model is not cached, initialize the ChromaDB searcher and load the model
    searcher = ChromaDBSearcher()  # Initialize the ChromaDB searcher for document retrieval
    model_qa = ModelQA(model_id=model_id, searcher=searcher)  # Create a new ModelQA instance
    model_cache.set(model_id, model_qa)  # Cache the model for future use

# Load the tokenizer and language model only once for efficiency
tokenizer, llm_model = model_qa.load_model()

/home/vinoj/.local/lib/python3.11/site-packages/transformers/models/auto/tokenization_auto.py:809: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/vinoj/.local/lib/python3.11/site-packages/transformers/models/auto/configuration_auto.py:1002: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/vinoj/.local/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# 3. Function to display chat history in a readable format
def display_chat_history(chat_history):
    """
    Display the chat history in the notebook with user and bot messages.

    Args:
    - chat_history (list): List of chat exchanges, each containing a user message and a bot response.
    """
    for chat in chat_history:
        # Sanitize and clean user and bot messages to remove unwanted HTML or special characters
        user_message = re.sub(r'<.*?>', '', chat["user"]).replace("~", "")
        bot_message = re.sub(r'<.*?>', '', chat["bot"]).replace("~", "")
        
        # Display user and bot messages in the notebook
        print(f"User: {user_message}")
        print(f"Bot: {bot_message}")
        print("-" * 50)

In [4]:
# 4. Initialize chat history list
chat_history = []

# 5. Available document sources (this could represent manuals, documents, etc.)
document_sources = [
    "Fraggles_X500_2024_FMS",
    "Fraggles_X700_2022_HCM"   
    # Add more sources as needed
]

In [5]:
# 6. Ask the user to select a document source from the available list
print("Select a Car Model, Make & Year from the list below:")
for i, doc in enumerate(document_sources):
    print(f"{i + 1}. {doc}")

# Get user input for selecting a document source
document_source_index = int(input("Enter the number corresponding to the document source: ")) - 1
document_source = document_sources[document_source_index]  # Get the selected document


Select a Car Model, Make & Year from the list below:
1. Fraggles_X500_2024_FMS
2. Fraggles_X700_2022_HCM


Enter the number corresponding to the document source:  1


In [6]:
# 7. Main loop for chatting with the bot
while True:
    # Prompt user for a query
    query = input("\nEnter your query here (or type 'exit' to quit): ")
    
    if query.lower() == 'exit':
        # Exit the chat loop if user types 'exit'
        print("Exiting the chat. Goodbye!")
        break

    # Process the query and get the response from the model
    print("\nGenerating response...")
    answer = model_qa.ask(document_source, query)  # Model will use the document source to generate the response
    
    # Display the bot's response
    
    #print(f"Bot: {answer}")
    
    # Append the user query and bot response to the chat history
    chat_history.append({"user": query, "bot": answer})

    # Optionally, display the full chat history
    display_chat_history(chat_history)


Enter your query here (or type 'exit' to quit):  contact number



Generating response...
User: contact number
Bot:  Using the following context items, please answer the user query directly.
                        Extract and incorporate relevant information from the context, but do not mention the context or how you arrived at your answer.
                        Provide a clear, concise, and explanatory answer.
                        
                        Now use the following context items to answer the user query:
                        - ['FragglesX500FMS 2024          1                                      Contact Information  If you require assistance or clariﬁcation on policies or  procedures, please contact the  customer relationship center. United States:      Fraggles Cars Auto  Customer Relationship Center  PO Box 12345  Brookﬁeld, NY 12345  Phone: 1-800-555-1234  Website: www.FragglesAutoSupport.com', 'FragglesX500FMS 2024          95  accept the decision, Fraggles must comply within 30  days. For assistance, contact AutoLine at 1-


Enter your query here (or type 'exit' to quit):  exit


Exiting the chat. Goodbye!
